# TFG - Anàlisi de l’abandonament escolar prematur a Catalunya
## Tractament de Dades Any 2021
Jahziel Hidalgo Artigas



## Taula de continguts<a class="anchor" id="index"></a>
* [Imports](#imports)
* [Funcions](#funcions)
* [Càrrega Datasets](#datasets)
* [Comarques](#comarques)
* [Dades Formació](#formacio)
    * [Anàlisi per Gènere](#formacio_genere)
    * [Formacio Gènere per Categoría](#formacio_categoria)
    * [Formació d'Adults](#formacio_adults)
* [Dades Població](#poblacio)
   * [Població per Gènere](#poblacio_genere)
   * [Població per Edat](#poblacio_edat)
   * [Població Activa (16-24 anys)](#poblacio_activa)
   * [Població Inactiva](#poblacio_inactiva)
   * [Percentatge/Ratio Estudiants](#pct_estudiants)
   * [Població per Nacionalitat](#poblacio_nacionalitat)
* [Dades Socioeconòmiques](#dades_socioeconomiques)
    * [Repetidors](#repetidors)
    * [Total Repetidors per Comarca](#repetidors_comarca)
    * [Atur per Edat](#atur)
    * [Atur per Gènere](#atur_genere)
    * [Impacte de la Nacionalitat](#extrangers)
    * [Nivell Socioeconòmic (Renda Familiar)](#renta_familiar)
    * [Nivell Socioeconòmic (RFBD](#rfbd)


### Imports<a class="anchor" id="imports"></a>

In [401]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import os
from unidecode import unidecode

### Funcions<a class="anchor" id="funcions"></a>

In [404]:
def clean_dataframe(df):
    # Netejar les columnes: treure els accents, substituir els espais per guions baixos i convertir a minúscules
    df.columns = [unidecode(col.replace(' ', '_').lower()) for col in df.columns]
    
    # Netejar les dades: treure els accents, convertir a minúscules i processar només les cadenes de text
    df = df.apply(lambda x: x.apply(lambda val: unidecode(str(val)).lower() if isinstance(val, str) else val) 
                  if x.dtype == 'object' else x, axis=0)
    
    return df

def filtrar_comarques(df, columna):
    comarques_valides = [
        'alt camp', 'alt emporda', 'alt penedes', 'alt urgell', 'alta ribagorca', 
        'anoia', 'aran', 'bages', 'baix camp', 'baix ebre', 'baix emporda', 
        'baix llobregat', 'baix penedes', 'barcelones', 'bergueda', 'cerdanya', 
        'conca de barbera', 'garraf', 'garrigues', 'garrotxa', 'girones', 'maresme', 
        'moianes', 'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira', 
        "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles', 
        'segarra', 'segria', 'selva', 'solsones', 'tarragones', 'terra alta', 'urgell', 
        'valles occidental', 'valles oriental'
    ]
    
    # Filtrar les files que continguin valors en la columna dins de la llista de comarques vàlides
    df = df[df[columna].isin(comarques_valides)].reset_index(drop=True)
    
    return df


### Càrrega Datasets<a class="anchor" id="datasets"></a>

In [407]:
df_comarques = pd.read_csv('csv/2021/comarques.csv', delimiter=';', skiprows=3) # Filtar per Nivell = 'Comarca'
df_formacio_sexe = pd.read_csv('csv/2021/nivell_formacio_sexe_2021.csv', delimiter=';') 
df_formacio_adults = pd.read_csv('csv/2021/formacio_adults_2021.csv', delimiter=';', skiprows=5)
df_poblacio_genere = pd.read_csv('csv/2021/poblacio.csv', delimiter=';') # Filtrar per any i poblacio total
df_poblacio_edat = pd.read_csv('csv/2021/poblacio_edat.csv', delimiter=';')
df_poblacio_activa = pd.read_csv('csv/2021/poblacio_activa_2021.csv', delimiter=';', skiprows=6)
df_poblacio_inactiva = pd.read_csv('csv/2021/poblacio_inactiva2021.csv', delimiter=';', skiprows=5)
df_poblacio_nacionalitat = pd.read_csv('csv/2021/poblacio_nacionalitat_2021.csv', delimiter=';', skiprows=7)
df_repetidors = pd.read_csv('csv/2021/alumnes_repetidors_2021.csv', delimiter=';')
df_renta_familiar = pd.read_csv('csv/2021/renta_familiar2021.csv', delimiter=';', skiprows=7)
df_rfbd = pd.read_csv('csv/2021/RFBD_2021.csv', delimiter=';', skiprows=7)
df_atur_edat = pd.read_csv('csv/2021/atur_edat_2021.csv', delimiter=';', skiprows=7)
df_atur_genere = pd.read_csv('csv/2021/atur_genere_2021.csv', delimiter=';', skiprows=6)

In [8]:
datasets = {
    'Comarques': df_comarques,
    'Formació Sexe': df_formacio_sexe,
    'Formació Adults': df_formacio_adults,
    'Població Sexe': df_poblacio_genere,
    'Població Edat': df_poblacio_edat,
    'Població Activa': df_poblacio_activa,
    'Població Inactiva': df_poblacio_inactiva,
    'Població Nacionalitat': df_poblacio_nacionalitat,
    'Repetidors': df_repetidors,
    'Renta Familiar': df_renta_familiar,
    'RFBD': df_rfbd
}

# Iterar sobre el diccionari per mostrar les primeres files de cada dataset
for name, df in datasets.items():
    print(f"Primeres files de {name}:")
    display(df.head()) 
    print("\n") 

Primeres files de Comarques:


,Nivell,Codi,Nom
0,Comarca,1,Alt Camp
1,Municipi,430017,Aiguamúrcia
2,Municipi,430056,Alcover
3,Municipi,430108,Alió
4,Municipi,430347,Bràfim




Primeres files de Formació Sexe:


,any,comarca o Aran,sexe,nivell de formació assolit,concepte,estat,valor
0,2021,Alt Camp,homes,analfabetisme,població de 15 anys o més,NaN,27
1,2021,Alt Camp,homes,educació primària incompleta,població de 15 anys o més,NaN,567
2,2021,Alt Camp,homes,educació primària,població de 15 anys o més,NaN,2064
3,2021,Alt Camp,homes,primera etapa d’educació secundària i similar,població de 15 anys o més,NaN,7395
4,2021,Alt Camp,homes,"segona etapa d’educació secundària, amb orient...",població de 15 anys o més,NaN,1932




Primeres files de Formació Adults:


...

,Curs,Estudi,Codi àrea territorial,Àrea territorial,Codi comarca,Comarca,Codi municipi,Municipi,Codi districte,Naturalesa,Titularitat,Nivell,Sexe,Concert,Resultat de l'avaluació,Nombre alumnes
0,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,DONA,SI,Alumnes avaluats,3
1,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,DONA,SI,Promocionen al mes de juny,2
2,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,DONA,SI,Promocionen al mes de setembre,1
3,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,HOME,SI,Alumnes avaluats,7
4,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,HOME,SI,Promocionen al mes de juny,3




Primeres files de Renta Familiar:


,Unnamed: 0,RFDB (miles de euros),RFDB por habitante (euros),RFDB por habitante (Índex Catalunya=100 por habitante)
0,Alt Camp,747053,16512,"90,0"
1,Alt Empordà,2059702,14339,"78,1"
2,Alt Penedès,1958312,17716,"96,5"
3,Alt Urgell,293835,14416,"78,5"
4,Alta Ribagorça,60948,15345,"83,6"




Primeres files de RFBD:


,Unnamed: 0,Recursos. Remuneración asalariados,Recursos. Excedente bruto explotación,Recursos. Prestaciones sociales,Usos. Cotizaciones sociales,Usos. Impuestos
0,Alt Camp,593173,173872,240921,179399,96508
1,Alt Empordà,1583386,657018,598935,512052,329330
2,Alt Penedès,1652972,474524,592603,490413,277769
3,Alt Urgell,197416,91423,97949,63921,36509
4,Alta Ribagorça,42647,22887,18521,14839,8501


### Comarques<a class="anchor" id="comarques"></a>

L'estudi es farà per comarques, així que necessitem obtenir el codi de les comarques, el qual ens servirà per agrupar el conjunt final.

In [11]:
# Primerament, netegem el dataset per assegurar-nos que no hi hagi problemes amb els noms de les columnes, accents, etc.
df_comarques = clean_dataframe(df_comarques)

In [12]:
df_comarques.head()

,nivell,codi,nom
0,comarca,1,alt camp
1,municipi,430017,aiguamurcia
2,municipi,430056,alcover
3,municipi,430108,alio
4,municipi,430347,brafim


In [13]:
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nivell  990 non-null    object
 1   codi    990 non-null    int64 
 2   nom     990 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.3+ KB


In [14]:
# Filtrarem les dades per seleccionar només aquelles files on el 'nivell' sigui 'comarca',
# i només seleccionarem les columnes 'codi' i 'nom' que seran necessàries per a l'anàlisi posterior.
df_comarques = df_comarques[df_comarques['nivell'] == 'comarca'][['codi', 'nom']].reset_index(drop=True)
# Convertim la columna 'codi' a format de dos dígits
df_comarques['codi'] = df_comarques['codi'].astype(str).str.zfill(2)
df_comarques.head()

,codi,nom
0,01,alt camp
1,02,alt emporda
2,03,alt penedes
3,04,alt urgell
4,05,alta ribagorca


In [15]:
# Afegim manualment el codi d'Aran (39)
df_comarques.loc[len(df_comarques)] = ['39', 'aran']
df_comarques = df_comarques.sort_values(by='codi').reset_index(drop=True)
df_comarques.tail()

,codi,nom
38,39,aran
39,40,valles occidental
40,41,valles oriental
41,42,moianes
42,43,llucanes


In [16]:
df_comarques['codi'] = df_comarques['codi'].astype(str)
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   codi    43 non-null     object
 1   nom     43 non-null     object
dtypes: object(2)
memory usage: 820.0+ bytes


In [17]:
# Comprovem els noms únics de les comarques per assegurar-nos que no hi ha comarques duplicades
df_comarques['nom'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'bages', 'baix camp', 'baix ebre',
       'baix emporda', 'baix llobregat', 'baix penedes', 'barcelones',
       'bergueda', 'cerdanya', 'conca de barbera', 'garraf', 'garrigues',
       'garrotxa', 'girones', 'maresme', 'montsia', 'noguera', 'osona',
       'pallars jussa', 'pallars sobira', "pla d'urgell",
       "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles',
       'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'aran', 'valles occidental',
       'valles oriental', 'moianes', 'llucanes'], dtype=object)

In [18]:
# Imprimim el nombre de comarques úniques que tenim en el dataset
print(f"Hi han {len(df_comarques['nom'].unique())} comarques")

Hi han 43 comarques


In [19]:
# Crear el diccionari de codis (comarca -> codi)
comarques_dict = df_comarques.set_index('nom')['codi'].to_dict()
comarques_dict

{'alt camp': '01',
 'alt emporda': '02',
 'alt penedes': '03',
 'alt urgell': '04',
 'alta ribagorca': '05',
 'anoia': '06',
 'bages': '07',
 'baix camp': '08',
 'baix ebre': '09',
 'baix emporda': '10',
 'baix llobregat': '11',
 'baix penedes': '12',
 'barcelones': '13',
 'bergueda': '14',
 'cerdanya': '15',
 'conca de barbera': '16',
 'garraf': '17',
 'garrigues': '18',
 'garrotxa': '19',
 'girones': '20',
 'maresme': '21',
 'montsia': '22',
 'noguera': '23',
 'osona': '24',
 'pallars jussa': '25',
 'pallars sobira': '26',
 "pla d'urgell": '27',
 "pla de l'estany": '28',
 'priorat': '29',
 "ribera d'ebre": '30',
 'ripolles': '31',
 'segarra': '32',
 'segria': '33',
 'selva': '34',
 'solsones': '35',
 'tarragones': '36',
 'terra alta': '37',
 'urgell': '38',
 'aran': '39',
 'valles occidental': '40',
 'valles oriental': '41',
 'moianes': '42',
 'llucanes': '43'}

### Dades Formació<a class="anchor" id="formacio"></a>

#### Anàlisi per Gènere<a class="anchor" id="formacio_genere"></a>

In [22]:
df_formacio_sexe = pd.read_csv('csv/2021/nivell_formacio_sexe_2021.csv', delimiter=';') 

In [23]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_genere_clean = clean_dataframe(df_formacio_sexe)
df_formacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   any                         1548 non-null   int64  
 1   comarca_o_aran              1548 non-null   object 
 2   sexe                        1548 non-null   object 
 3   nivell_de_formacio_assolit  1548 non-null   object 
 4   concepte                    1548 non-null   object 
 5   estat                       0 non-null      float64
 6   valor                       1548 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 84.8+ KB


In [24]:
# Corregir nom columna comarca i filtrar per 'total' i només per l'any 2021
df_formacio_genere_clean.rename(columns={'comarca_o_aran': 'comarca'}, inplace=True)
df_formacio_genere_clean = df_formacio_genere_clean[df_formacio_genere_clean['nivell_de_formacio_assolit'] != 'total']
df_formacio_genere_clean = df_formacio_genere_clean[df_formacio_genere_clean['any'] == 2021]

In [25]:
# Filtrar per comarques
df_formacio_genere_clean = filtrar_comarques(df_formacio_genere_clean, 'comarca')

In [26]:
# Comprovem els canvis realitzats
df_formacio_genere_clean.info()
display(df_formacio_genere_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   any                         1386 non-null   int64  
 1   comarca                     1386 non-null   object 
 2   sexe                        1386 non-null   object 
 3   nivell_de_formacio_assolit  1386 non-null   object 
 4   concepte                    1386 non-null   object 
 5   estat                       0 non-null      float64
 6   valor                       1386 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 75.9+ KB


,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor
0,2021,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27
1,2021,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,567
2,2021,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,2064
3,2021,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7395
4,2021,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1932
5,2021,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,2361
6,2021,alt camp,homes,ensenyaments de grau superior de formacio prof...,poblacio de 15 anys o mes,NaN,2265
7,2021,alt camp,homes,graus universitaris de fins a 240 credits i si...,poblacio de 15 anys o mes,NaN,1302
8,2021,alt camp,homes,graus universitaris de mes de 240 credits i si...,poblacio de 15 anys o mes,NaN,1035
9,2021,alt camp,homes,masters universitaris i similars,poblacio de 15 anys o mes,NaN,327


In [28]:
# Creem el mapeig de categories educatives
categoria_map = {
    'analfabetisme': 'analfabetisme',
    'educacio primaria incompleta': 'educacio primaria incompleta',
    'educacio primaria': 'educacio primaria',
    'primera etapa d\'educacio secundaria i similar': 'educacio secundaria',
    'segona etapa d\'educacio secundaria, amb orientacio general': 'educacio secundaria',
    'segona etapa d\'educacio secundaria, amb orientacio professional': 'educacio secundaria',
    'ensenyaments de grau superior de formacio professional i similars': 'formacio professional',
    'graus universitaris de fins a 240 credits i similars': 'graus universitaris',
    'graus universitaris de mes de 240 credits i similars': 'graus universitaris',
    'masters universitaris i similars': 'masters universitaris',
    'doctorat universitari': 'doctorat universitari'
}
# Assignem la categoria a cada nivell de formació
df_formacio_genere_clean['categoria_educacio'] = df_formacio_genere_clean['nivell_de_formacio_assolit'].map(categoria_map)
df_formacio_genere_clean.head()

,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor,categoria_educacio
0,2021,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27,analfabetisme
1,2021,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,567,educacio primaria incompleta
2,2021,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,2064,educacio primaria
3,2021,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7395,educacio secundaria
4,2021,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1932,educacio secundaria


In [29]:
df_formacio_genere_clean['categoria_educacio'].unique()

array(['analfabetisme', 'educacio primaria incompleta',
       'educacio primaria', 'educacio secundaria',
       'formacio professional', 'graus universitaris',
       'masters universitaris', 'doctorat universitari'], dtype=object)

In [30]:
# Seleccionem les columnes necessàries
df_formacio_grouped = df_formacio_genere_clean[['comarca', 'sexe', 'valor', 'categoria_educacio']]

In [31]:
# Agrupem les dades per comarca, categoria d'educació i sexe
df_grouped = df_formacio_grouped.groupby(['comarca', 'categoria_educacio', 'sexe'])['valor'].sum().reset_index()

In [32]:
# Comprovem les dades agrupades
df_grouped.head(10)

,comarca,categoria_educacio,sexe,valor
0,alt camp,analfabetisme,dones,60
1,alt camp,analfabetisme,homes,27
2,alt camp,analfabetisme,total,87
3,alt camp,doctorat universitari,dones,120
4,alt camp,doctorat universitari,homes,102
5,alt camp,doctorat universitari,total,219
6,alt camp,educacio primaria,dones,2367
7,alt camp,educacio primaria,homes,2064
8,alt camp,educacio primaria,total,4428
9,alt camp,educacio primaria incompleta,dones,783


In [33]:
# Creem una taula pivote per a cada combinació de categoria i sexe
df_formacio_genere = df_grouped.pivot_table(
    index='comarca', 
    columns=['categoria_educacio', 'sexe'], 
    values='valor', 
    aggfunc='sum'
).reset_index()
df_formacio_genere.columns = df_formacio_genere.columns.to_series().str.join('_')
df_formacio_genere.rename(columns={'comarca_': 'comarca'}, inplace=True)
df_formacio_genere.head()

,comarca,analfabetisme_dones,analfabetisme_homes,analfabetisme_total,doctorat universitari_dones,doctorat universitari_homes,doctorat universitari_total,educacio primaria_dones,educacio primaria_homes,educacio primaria_total,...,educacio secundaria_total,formacio professional_dones,formacio professional_homes,formacio professional_total,graus universitaris_dones,graus universitaris_homes,graus universitaris_total,masters universitaris_dones,masters universitaris_homes,masters universitaris_total
0,alt camp,60,27,87,120,102,219,2367,2064,4428,...,21675,1848,2265,4110,3177,2337,5517,489,327,813
1,alt emporda,210,129,336,312,402,714,9276,8577,17850,...,69471,3567,3867,7431,10089,7350,17439,1206,858,2064
2,alt penedes,141,39,177,324,348,672,6471,5181,11655,...,50148,4440,4968,9411,9108,6666,15777,1215,774,1989
3,alt urgell,18,12,30,72,78,147,1158,1017,2175,...,9708,672,831,1506,1872,1218,3093,300,183,486
4,alta ribagorca,6,3,9,15,15,33,231,174,402,...,1725,150,198,348,435,294,732,48,30,81


In [34]:
df_formacio_genere.comarca.unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental'],
      dtype=object)

In [35]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_genere.insert(0, 'codi', df_formacio_genere['comarca'].map(comarques_dict))
df_formacio_genere = df_formacio_genere.sort_values(by='codi').reset_index(drop=True)

In [36]:
df_formacio_genere.head()

,codi,comarca,analfabetisme_dones,analfabetisme_homes,analfabetisme_total,doctorat universitari_dones,doctorat universitari_homes,doctorat universitari_total,educacio primaria_dones,educacio primaria_homes,...,educacio secundaria_total,formacio professional_dones,formacio professional_homes,formacio professional_total,graus universitaris_dones,graus universitaris_homes,graus universitaris_total,masters universitaris_dones,masters universitaris_homes,masters universitaris_total
0,01,alt camp,60,27,87,120,102,219,2367,2064,...,21675,1848,2265,4110,3177,2337,5517,489,327,813
1,02,alt emporda,210,129,336,312,402,714,9276,8577,...,69471,3567,3867,7431,10089,7350,17439,1206,858,2064
2,03,alt penedes,141,39,177,324,348,672,6471,5181,...,50148,4440,4968,9411,9108,6666,15777,1215,774,1989
3,04,alt urgell,18,12,30,72,78,147,1158,1017,...,9708,672,831,1506,1872,1218,3093,300,183,486
4,05,alta ribagorca,6,3,9,15,15,33,231,174,...,1725,150,198,348,435,294,732,48,30,81


In [37]:
df_formacio_genere.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   codi                                42 non-null     object
 1   comarca                             42 non-null     object
 2   analfabetisme_dones                 42 non-null     int64 
 3   analfabetisme_homes                 42 non-null     int64 
 4   analfabetisme_total                 42 non-null     int64 
 5   doctorat universitari_dones         42 non-null     int64 
 6   doctorat universitari_homes         42 non-null     int64 
 7   doctorat universitari_total         42 non-null     int64 
 8   educacio primaria_dones             42 non-null     int64 
 9   educacio primaria_homes             42 non-null     int64 
 10  educacio primaria_total             42 non-null     int64 
 11  educacio primaria incompleta_dones  42 non-null     int64 
 

In [38]:
df_formacio_genere.to_csv('clean_data/2021/df_formacio_genere.csv', index=False)

### Formacio Gènere per Categoría<a class="anchor" id="formacio_categoria"></a>

In [462]:
df_formacio_sexe = pd.read_csv('csv/2021/nivell_formacio_sexe_2021.csv', delimiter=';') 

In [464]:
df_formacio_sexe

,any,comarca o Aran,sexe,nivell de formació assolit,concepte,estat,valor
0,2021,Alt Camp,homes,analfabetisme,població de 15 anys o més,NaN,27
1,2021,Alt Camp,homes,educació primària incompleta,població de 15 anys o més,NaN,567
2,2021,Alt Camp,homes,educació primària,població de 15 anys o més,NaN,2064
3,2021,Alt Camp,homes,primera etapa d’educació secundària i similar,població de 15 anys o més,NaN,7395
4,2021,Alt Camp,homes,"segona etapa d’educació secundària, amb orient...",població de 15 anys o més,NaN,1932
...,...,...,...,...,...,...,...
1543,2021,Catalunya,total,graus universitaris de fins a 240 crèdits i si...,població de 15 anys o més,NaN,718070
1544,2021,Catalunya,total,graus universitaris de més de 240 crèdits i si...,població de 15 anys o més,NaN,639256
1545,2021,Catalunya,total,màsters universitaris i similars,població de 15 anys o més,NaN,189056
1546,2021,Catalunya,total,doctorat universitari,població de 15 anys o més,NaN,77913


In [466]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_genere_clean = clean_dataframe(df_formacio_sexe)
df_formacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   any                         1548 non-null   int64  
 1   comarca_o_aran              1548 non-null   object 
 2   sexe                        1548 non-null   object 
 3   nivell_de_formacio_assolit  1548 non-null   object 
 4   concepte                    1548 non-null   object 
 5   estat                       0 non-null      float64
 6   valor                       1548 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 84.8+ KB


In [468]:
# Corregir nom columna comarca i filtrar per 'total' i només per l'any 2021
df_formacio_genere_clean.rename(columns={'comarca_o_aran': 'comarca'}, inplace=True)
df_formacio_genere_clean = df_formacio_genere_clean[df_formacio_genere_clean['nivell_de_formacio_assolit'] != 'total']
df_formacio_genere_clean = df_formacio_genere_clean[df_formacio_genere_clean['any'] == 2021]

In [470]:
# Filtrar per comarques
df_formacio_genere_clean = filtrar_comarques(df_formacio_genere_clean, 'comarca')

In [472]:
# Comprovem els canvis realitzats
df_formacio_genere_clean.info()
display(df_formacio_genere_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   any                         1386 non-null   int64  
 1   comarca                     1386 non-null   object 
 2   sexe                        1386 non-null   object 
 3   nivell_de_formacio_assolit  1386 non-null   object 
 4   concepte                    1386 non-null   object 
 5   estat                       0 non-null      float64
 6   valor                       1386 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 75.9+ KB


,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor
0,2021,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27
1,2021,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,567
2,2021,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,2064
3,2021,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7395
4,2021,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1932
5,2021,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,2361
6,2021,alt camp,homes,ensenyaments de grau superior de formacio prof...,poblacio de 15 anys o mes,NaN,2265
7,2021,alt camp,homes,graus universitaris de fins a 240 credits i si...,poblacio de 15 anys o mes,NaN,1302
8,2021,alt camp,homes,graus universitaris de mes de 240 credits i si...,poblacio de 15 anys o mes,NaN,1035
9,2021,alt camp,homes,masters universitaris i similars,poblacio de 15 anys o mes,NaN,327


In [474]:
df_formacio_genere_clean['nivell_de_formacio_assolit'].unique()

array(['analfabetisme', 'educacio primaria incompleta',
       'educacio primaria',
       "primera etapa d'educacio secundaria i similar",
       "segona etapa d'educacio secundaria, amb orientacio general",
       "segona etapa d'educacio secundaria, amb orientacio professional",
       'ensenyaments de grau superior de formacio professional i similars',
       'graus universitaris de fins a 240 credits i similars',
       'graus universitaris de mes de 240 credits i similars',
       'masters universitaris i similars', 'doctorat universitari'],
      dtype=object)

Agrupem els següents grups per homes, dones i total:  
* sense educació
    * analfabetisme
    * educació primària incompleta  
* educació obligatoria
    *  educació primaria
    *  primera etapa d’educació secundària i similarmària
    *  segona etapa d’educació secundària, amb orientació general
    *  segona etapa d’educació secundària, amb orientació professionales
* estudis superiors
    * ensenyaments de grau superior de formació professional i similars
    * graus universitaris de fins a 240 crèdits i similars
    * graus universitaris de més de 240 crèdits i similars
    * màsters universitaris i similars
    * doctorat universitari

In [476]:
# Creem el mapeig de categories educatives
categoria_map = {
    'analfabetisme': 'sense_educacio',
    'educacio primaria incompleta': 'sense_educacio',
    'educacio primaria': 'educacio_obligatoria',
    'primera etapa d\'educacio secundaria i similar': 'educacio_obligatoria',
    'segona etapa d\'educacio secundaria, amb orientacio general': 'educacio_obligatoria',
    'segona etapa d\'educacio secundaria, amb orientacio professional': 'educacio_obligatoria',
    'ensenyaments de grau superior de formacio professional i similars': 'estudis_superiors',
    'graus universitaris de fins a 240 credits i similars': 'estudis_superiors',
    'graus universitaris de mes de 240 credits i similars': 'estudis_superiors',
    'masters universitaris i similars': 'estudis_superiors',
    'doctorat universitari': 'estudis_superiors'
}
# Assignem la categoria a cada nivell de formació
df_formacio_genere_clean['tipus_estudis'] = df_formacio_genere_clean['nivell_de_formacio_assolit'].map(categoria_map)
df_formacio_genere_clean.head()

,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor,tipus_estudis
0,2021,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27,sense_educacio
1,2021,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,567,sense_educacio
2,2021,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,2064,educacio_obligatoria
3,2021,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7395,educacio_obligatoria
4,2021,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1932,educacio_obligatoria


In [478]:
df_formacio_genere_clean

,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor,tipus_estudis
0,2021,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27,sense_educacio
1,2021,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,567,sense_educacio
2,2021,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,2064,educacio_obligatoria
3,2021,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7395,educacio_obligatoria
4,2021,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1932,educacio_obligatoria
...,...,...,...,...,...,...,...,...
1381,2021,valles oriental,total,ensenyaments de grau superior de formacio prof...,poblacio de 15 anys o mes,NaN,37854,estudis_superiors
1382,2021,valles oriental,total,graus universitaris de fins a 240 credits i si...,poblacio de 15 anys o mes,NaN,34299,estudis_superiors
1383,2021,valles oriental,total,graus universitaris de mes de 240 credits i si...,poblacio de 15 anys o mes,NaN,26718,estudis_superiors
1384,2021,valles oriental,total,masters universitaris i similars,poblacio de 15 anys o mes,NaN,7770,estudis_superiors


In [480]:
df_formacio_genere_clean['tipus_estudis'].unique()

array(['sense_educacio', 'educacio_obligatoria', 'estudis_superiors'],
      dtype=object)

In [442]:
# Seleccionem les columnes necessàries
df_formacio_grouped = df_formacio_genere_clean[['comarca', 'sexe', 'valor', 'tipus_estudis']]

In [482]:
# Agrupem les dades per comarca, categoria d'educació i sexe
df_formacio_grouped = df_formacio_grouped.groupby(['comarca', 'tipus_estudis', 'sexe'])['valor'].sum().reset_index()
df_formacio_grouped.head(10)

,comarca,tipus_estudis,sexe,valor
0,alt camp,educacio_obligatoria,dones,12360
1,alt camp,educacio_obligatoria,homes,13752
2,alt camp,educacio_obligatoria,total,26103
3,alt camp,estudis_superiors,dones,5634
4,alt camp,estudis_superiors,homes,5031
5,alt camp,estudis_superiors,total,10659
6,alt camp,sense_educacio,dones,843
7,alt camp,sense_educacio,homes,594
8,alt camp,sense_educacio,total,1437
9,alt emporda,educacio_obligatoria,dones,42108


In [484]:
# Eliminar files on 'sexe' es igual a 'total'
df_grouped_genere = df_formacio_grouped[df_formacio_grouped['sexe'] != 'total']
df_grouped_genere.head()

,comarca,tipus_estudis,sexe,valor
0,alt camp,educacio_obligatoria,dones,12360
1,alt camp,educacio_obligatoria,homes,13752
3,alt camp,estudis_superiors,dones,5634
4,alt camp,estudis_superiors,homes,5031
6,alt camp,sense_educacio,dones,843


In [492]:
df_grouped_genere.head(20)

,comarca,tipus_estudis,sexe,valor
0,alt camp,educacio_obligatoria,dones,12360
1,alt camp,educacio_obligatoria,homes,13752
3,alt camp,estudis_superiors,dones,5634
4,alt camp,estudis_superiors,homes,5031
6,alt camp,sense_educacio,dones,843
7,alt camp,sense_educacio,homes,594
9,alt emporda,educacio_obligatoria,dones,42108
10,alt emporda,educacio_obligatoria,homes,45210
12,alt emporda,estudis_superiors,dones,15174
13,alt emporda,estudis_superiors,homes,12477


In [494]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_grouped_genere.insert(0, 'codi', df_grouped_genere['comarca'].map(comarques_dict))
df_grouped_genere = df_grouped_genere.sort_values(by='codi').reset_index(drop=True)
df_grouped_genere.head()

,codi,comarca,tipus_estudis,sexe,valor
0,01,alt camp,educacio_obligatoria,dones,12360
1,01,alt camp,educacio_obligatoria,homes,13752
2,01,alt camp,estudis_superiors,dones,5634
3,01,alt camp,estudis_superiors,homes,5031
4,01,alt camp,sense_educacio,dones,843


In [496]:
df_grouped_genere.head(30)

,codi,comarca,tipus_estudis,sexe,valor
0,01,alt camp,educacio_obligatoria,dones,12360
1,01,alt camp,educacio_obligatoria,homes,13752
2,01,alt camp,estudis_superiors,dones,5634
3,01,alt camp,estudis_superiors,homes,5031
4,01,alt camp,sense_educacio,dones,843
5,01,alt camp,sense_educacio,homes,594
6,02,alt emporda,educacio_obligatoria,dones,42108
7,02,alt emporda,educacio_obligatoria,homes,45210
8,02,alt emporda,estudis_superiors,dones,15174
9,02,alt emporda,estudis_superiors,homes,12477


In [498]:
df_grouped_genere.to_csv('clean_data/2021/formacio_genere_pivot.csv', index=False)

#### Formació d'Adults<a class="anchor" id="formacio_adults"></a>

In [40]:
df_formacio_adults = pd.read_csv('csv/formacio_adults.csv', delimiter=';', skiprows=5)

In [41]:
df_formacio_adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    43 non-null     object
 1   Centres                    43 non-null     int64 
 2   Personal docent            43 non-null     int64 
 3   Alumnes (homes). 16-24     43 non-null     int64 
 4   Alumnes (homes). 25-64     43 non-null     int64 
 5   Alumnes (homes). 65 i més  43 non-null     int64 
 6   Alumnes (homes). Total     43 non-null     int64 
 7   Alumnes (dones). 16-24     43 non-null     int64 
 8   Alumnes (dones). 25-64     43 non-null     int64 
 9   Alumnes (dones). 65 i més  43 non-null     int64 
 10  Alumnes (dones). Total     43 non-null     int64 
 11  Alumnes (total). 16-24     43 non-null     int64 
 12  Alumnes (total). 25-64     43 non-null     int64 
 13  Alumnes (total). 65 i més  43 non-null     int64 
 14  Alumnes (tot

In [42]:
df_formacio_adults.head(10)

,comarca,Centres,Personal docent,Alumnes (homes). 16-24,Alumnes (homes). 25-64,Alumnes (homes). 65 i més,Alumnes (homes). Total,Alumnes (dones). 16-24,Alumnes (dones). 25-64,Alumnes (dones). 65 i més,Alumnes (dones). Total,Alumnes (total). 16-24,Alumnes (total). 25-64,Alumnes (total). 65 i més,Alumnes (total). Total
0,Alt Camp,1,7,24,59,8,91,24,155,34,213,48,214,42,304
1,Alt Empordà,4,38,235,616,12,863,66,317,17,400,301,933,29,1263
2,Alt Penedès,2,13,82,124,18,224,52,217,22,291,134,341,40,515
3,Alt Urgell,1,6,19,103,18,140,23,221,37,281,42,324,55,421
4,Alta Ribagorça,1,1,0,0,0,0,0,20,0,20,0,20,0,20
5,Anoia,1,5,27,58,2,87,20,152,0,172,47,210,2,259
6,Aran,1,3,4,44,4,52,10,124,8,142,14,168,12,194
7,Bages,3,38,110,583,36,729,76,444,37,557,186,1027,73,1286
8,Baix Camp,4,38,142,266,39,447,118,684,78,880,260,950,117,1327
9,Baix Ebre,2,13,42,63,25,130,58,219,40,317,100,282,65,447


In [43]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_adults_clean = clean_dataframe(df_formacio_adults)
df_formacio_adults_clean.head()

,comarca,centres,personal_docent,alumnes_(homes)._16-24,alumnes_(homes)._25-64,alumnes_(homes)._65_i_mes,alumnes_(homes)._total,alumnes_(dones)._16-24,alumnes_(dones)._25-64,alumnes_(dones)._65_i_mes,alumnes_(dones)._total,alumnes_(total)._16-24,alumnes_(total)._25-64,alumnes_(total)._65_i_mes,alumnes_(total)._total
0,alt camp,1,7,24,59,8,91,24,155,34,213,48,214,42,304
1,alt emporda,4,38,235,616,12,863,66,317,17,400,301,933,29,1263
2,alt penedes,2,13,82,124,18,224,52,217,22,291,134,341,40,515
3,alt urgell,1,6,19,103,18,140,23,221,37,281,42,324,55,421
4,alta ribagorca,1,1,0,0,0,0,0,20,0,20,0,20,0,20


In [44]:
# Filtrar per comarques
df_formacio_adults_clean = filtrar_comarques(df_formacio_adults_clean, 'comarca')
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    42 non-null     object
 1   centres                    42 non-null     int64 
 2   personal_docent            42 non-null     int64 
 3   alumnes_(homes)._16-24     42 non-null     int64 
 4   alumnes_(homes)._25-64     42 non-null     int64 
 5   alumnes_(homes)._65_i_mes  42 non-null     int64 
 6   alumnes_(homes)._total     42 non-null     int64 
 7   alumnes_(dones)._16-24     42 non-null     int64 
 8   alumnes_(dones)._25-64     42 non-null     int64 
 9   alumnes_(dones)._65_i_mes  42 non-null     int64 
 10  alumnes_(dones)._total     42 non-null     int64 
 11  alumnes_(total)._16-24     42 non-null     int64 
 12  alumnes_(total)._25-64     42 non-null     int64 
 13  alumnes_(total)._65_i_mes  42 non-null     int64 
 14  alumnes_(tot

In [45]:
# Suma d'alumnes majors de 25 per a homes, dones i el total
df_formacio_adults_clean.loc[:, 'alumnes_homes_+25_anys'] = df_formacio_adults_clean['alumnes_(homes)._25-64'] + df_formacio_adults['alumnes_(homes)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_dones_+25_anys'] = df_formacio_adults_clean['alumnes_(dones)._25-64'] + df_formacio_adults['alumnes_(dones)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_total_+25_anys'] = df_formacio_adults_clean['alumnes_(total)._25-64'] + df_formacio_adults['alumnes_(total)._65_i_mes']


# Convertir les columnes +25 anys a tipus int
df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']] = (
    df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']].astype(int)
)

# Seleccionar i renombrar columnes
df_formacio_adults_clean = df_formacio_adults_clean[['comarca',
                                         'alumnes_(homes)._16-24',
                                         'alumnes_(dones)._16-24',
                                         'alumnes_(total)._16-24',
                                         'alumnes_homes_+25_anys',
                                         'alumnes_dones_+25_anys',
                                         'alumnes_total_+25_anys']].rename(columns={
    'alumnes_(homes)._16-24': 'alumnes_homes_16_24_anys',
    'alumnes_(dones)._16-24': 'alumnes_dones_16_24_anys',
    'alumnes_(total)._16-24': 'alumnes_total_16_24_anys'
})
# Dataset resultant
df_formacio_adults_clean.head()


,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,alt camp,24,24,48,67,189,256
1,alt emporda,235,66,301,628,334,962
2,alt penedes,82,52,134,142,239,381
3,alt urgell,19,23,42,121,258,379
4,alta ribagorca,0,0,0,0,20,20


In [46]:
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   comarca                   42 non-null     object
 1   alumnes_homes_16_24_anys  42 non-null     int64 
 2   alumnes_dones_16_24_anys  42 non-null     int64 
 3   alumnes_total_16_24_anys  42 non-null     int64 
 4   alumnes_homes_+25_anys    42 non-null     int32 
 5   alumnes_dones_+25_anys    42 non-null     int32 
 6   alumnes_total_+25_anys    42 non-null     int32 
dtypes: int32(3), int64(3), object(1)
memory usage: 1.9+ KB


In [47]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_adults_clean.insert(0, 'codi', df_formacio_adults_clean['comarca'].map(comarques_dict))
df_formacio_adults_clean = df_formacio_adults_clean.sort_values(by='codi').reset_index(drop=True)
df_formacio_adults_clean.head()

,codi,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,01,alt camp,24,24,48,67,189,256
1,02,alt emporda,235,66,301,628,334,962
2,03,alt penedes,82,52,134,142,239,381
3,04,alt urgell,19,23,42,121,258,379
4,05,alta ribagorca,0,0,0,0,20,20


In [49]:
df_formacio_adults_clean.to_csv('clean_data/2021/df_formacio_adults.csv', index=False)

### Dades Població<a class="anchor" id="poblacio"></a>

#### Població per Gènere<a class="anchor" id="poblacio_genere"></a>

In [355]:
df_poblacio_genere = pd.read_csv('csv/2021/poblacio.csv', delimiter=';') # Filtrar per any i poblacio total

In [357]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_poblacio_genere_clean = clean_dataframe(df_poblacio_genere)
df_poblacio_genere_clean.head()

,any,comarca_o_aran,sexe,concepte,estat,valor
0,1975,alt camp,homes,poblacio,NaN,15242.0
1,1975,alt camp,dones,poblacio,NaN,15869.0
2,1975,alt camp,total,poblacio,NaN,31111.0
3,1975,alt emporda,homes,poblacio,NaN,38608.0
4,1975,alt emporda,dones,poblacio,NaN,38993.0


In [359]:
df_poblacio_genere_clean['comarca_o_aran'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya'], dtype=object)

In [361]:
# Canviar el nom de la columna 'comarca_o_aran' a 'comarca' per simplificar el nom
df_poblacio_genere_clean.rename(columns={'comarca_o_aran': 'comarca'}, inplace=True)
# Filtrar el DataFrame per excloure les files on la comarca és 'catalunya', ja que només volem les dades a nivell de comarques individuals
df_poblacio_genere_clean = df_poblacio_genere_clean[df_poblacio_genere_clean['comarca'] != 'catalunya']
# Filtrar les dades per mantenir únicament les corresponents a l'any 2021
df_poblacio_genere_clean = df_poblacio_genere_clean[df_poblacio_genere_clean['any'] == 2021]

In [363]:
# Filtrem per comarques
df_poblacio_genere_clean = filtrar_comarques(df_poblacio_genere_clean, 'comarca')

In [365]:
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   any       126 non-null    int64  
 1   comarca   126 non-null    object 
 2   sexe      126 non-null    object 
 3   concepte  126 non-null    object 
 4   estat     0 non-null      object 
 5   valor     126 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 6.0+ KB


In [367]:
df_poblacio_genere_clean.head()

,any,comarca,sexe,concepte,estat,valor
0,2021,alt camp,homes,poblacio,NaN,22871.0
1,2021,alt camp,dones,poblacio,NaN,22207.0
2,2021,alt camp,total,poblacio,NaN,45078.0
3,2021,alt emporda,homes,poblacio,NaN,72029.0
4,2021,alt emporda,dones,poblacio,NaN,71368.0


In [369]:
# Crear una taula pivot amb 'comarca' com a índex, 'sexe' com a columnes i suma de 'valor'
df_poblacio_genere_clean = df_poblacio_genere_clean.pivot_table(
    index='comarca', 
    columns=['sexe'], 
    values='valor', 
    aggfunc='sum'
).reset_index()
# Renombrar les columnes a 'comarca', 'dones', 'homes', i 'total'
df_poblacio_genere_clean.columns = ['comarca', 'dones', 'homes', 'total']
df_poblacio_genere_clean.head()

,comarca,dones,homes,total
0,alt camp,22207.0,22871.0,45078.0
1,alt emporda,71368.0,72029.0,143397.0
2,alt penedes,55158.0,55040.0,110198.0
3,alt urgell,10207.0,10237.0,20444.0
4,alta ribagorca,1927.0,2033.0,3960.0


In [371]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_genere_clean.insert(0, 'codi', df_poblacio_genere_clean['comarca'].map(comarques_dict))
df_poblacio_genere_clean = df_poblacio_genere_clean.sort_values(by='codi').reset_index(drop=True)
# Convertir les columnes 'dones', 'homes' i 'total' a tipus int
df_poblacio_genere_clean[['dones', 'homes', 'total']] = df_poblacio_genere_clean[['dones', 'homes', 'total']].astype(int)
df_poblacio_genere_clean.head()

,codi,comarca,dones,homes,total
0,01,alt camp,22207,22871,45078
1,02,alt emporda,71368,72029,143397
2,03,alt penedes,55158,55040,110198
3,04,alt urgell,10207,10237,20444
4,05,alta ribagorca,1927,2033,3960


In [373]:
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   dones    42 non-null     int32 
 3   homes    42 non-null     int32 
 4   total    42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [375]:
df_poblacio_genere_clean.to_csv('clean_data/2021/df_poblacio_genere.csv', index=False)

#### Població per Edat<a class="anchor" id="poblacio_edat"></a>

In [64]:
df_poblacio_edat = pd.read_csv('csv/2021/poblacio_edat.csv', delimiter=';')

In [65]:
df_poblacio_edat.head() 

,any,comarca o Aran,edat,sexe,concepte,estat,valor
0,2011,Alt Camp,0 anys,homes,població,NaN,266.0
1,2011,Alt Camp,0 anys,dones,població,NaN,272.0
2,2011,Alt Camp,0 anys,total,població,NaN,538.0
3,2011,Alt Camp,1 any,homes,població,NaN,277.0
4,2011,Alt Camp,1 any,dones,població,NaN,252.0


In [66]:
# Netejar el DataFrame 'df_poblacio_edat' utilitzant la funció 'clean_dataframe'
df_poblacio_edat_clean = clean_dataframe(df_poblacio_edat)
df_poblacio_edat_clean.head()

,any,comarca_o_aran,edat,sexe,concepte,estat,valor
0,2011,alt camp,0 anys,homes,poblacio,NaN,266.0
1,2011,alt camp,0 anys,dones,poblacio,NaN,272.0
2,2011,alt camp,0 anys,total,poblacio,NaN,538.0
3,2011,alt camp,1 any,homes,poblacio,NaN,277.0
4,2011,alt camp,1 any,dones,poblacio,NaN,252.0


In [67]:
# Filtrar les comarques utilitzant la funció 'filtrar_comarques'
df_poblacio_edat_clean = filtrar_comarques(df_poblacio_edat_clean, 'comarca_o_aran')

In [68]:
# Renombrar la columna 'comarca_o_aran' a 'comarca'
df_poblacio_edat_clean.rename(columns={'comarca_o_aran': 'comarca'}, inplace=True)
# Filtrar les dades per mantenir només les corresponents a l'any 2021
df_poblacio_edat_clean = df_poblacio_edat_clean[df_poblacio_edat_clean['any'] == 2021]

In [69]:
df_poblacio_edat_clean.head()

,any,comarca,edat,sexe,concepte,estat,valor
12852,2021,alt camp,0 anys,homes,poblacio,NaN,179.0
12853,2021,alt camp,0 anys,dones,poblacio,NaN,172.0
12854,2021,alt camp,0 anys,total,poblacio,NaN,351.0
12855,2021,alt camp,1 any,homes,poblacio,NaN,190.0
12856,2021,alt camp,1 any,dones,poblacio,NaN,170.0


In [70]:
# Filtrar les dades per incloure només edats entre 16 i 24 anys
df_16_24 = df_poblacio_edat_clean[df_poblacio_edat_clean['edat'].isin(['16 anys', '17 anys', '18 anys', '19 anys', '20 anys', '21 anys', '22 anys', '23 anys', '24 anys'])]

# Agrupar les dades per 'comarca' i 'edat', sumant els valors
df_age_grouped = df_16_24.groupby(['comarca', 'edat'])['valor'].sum().reset_index()

# Crear una taula pivot amb 'comarca' com a índex i les edats com a columnes
df_pivoted = df_age_grouped.pivot_table(
    index='comarca', 
    columns='edat', 
    values='valor', 
    aggfunc='sum', 
    fill_value=0
)

# Afegir una columna amb la suma total de la població entre 16 i 24 anys per comarca
df_pivoted['poblacio_16_a_24_anys'] = df_pivoted.sum(axis=1)
df_pivoted.head()

edat,16 anys,17 anys,18 anys,19 anys,20 anys,21 anys,22 anys,23 anys,24 anys,poblacio_16_a_24_anys
comarca,,,,,,,,,,
alt camp,914.0,1036.0,958.0,908.0,1020.0,908.0,926.0,862.0,870.0,8402.0
alt emporda,3024.0,2992.0,2894.0,2964.0,2964.0,3000.0,2830.0,3010.0,2938.0,26616.0
alt penedes,2700.0,2538.0,2458.0,2444.0,2360.0,2168.0,2210.0,2110.0,1976.0,20964.0
alt urgell,400.0,354.0,342.0,348.0,400.0,420.0,408.0,388.0,404.0,3464.0
alta ribagorca,66.0,46.0,62.0,70.0,80.0,72.0,66.0,72.0,66.0,600.0


In [71]:
# Filtrar el DataFrame per la franja d'edat 16-24 anys i 'sexe' = 'total'
df_16_24_total = df_poblacio_edat_clean[(df_poblacio_edat_clean['edat'].isin(['16 anys', '17 anys', '18 anys', '19 anys', '20 anys', '21 anys', '22 anys', '23 anys', '24 anys'])) & (df_poblacio_edat_clean['sexe'] == 'total')]

# Agrupar per 'comarca' i sumar els valors per obtenir la població total
df_16_24_total_grouped = df_16_24_total.groupby('comarca')['valor'].sum().reset_index()
df_16_24_total_grouped.rename(columns={'valor': 'poblacio_total_16_a_24_anys'}, inplace=True)

# Filtrar el DataFrame per homes ('homes') i dones ('dones') dins la franja d'edat 16-24 anys
df_16_24_homes = df_poblacio_edat_clean[(df_poblacio_edat_clean['edat'].isin(['16 anys', '17 anys', '18 anys', '19 anys', '20 anys', '21 anys', '22 anys', '23 anys', '24 anys'])) & (df_poblacio_edat_clean['sexe'] == 'homes')]
df_16_24_dones = df_poblacio_edat_clean[(df_poblacio_edat_clean['edat'].isin(['16 anys', '17 anys', '18 anys', '19 anys', '20 anys', '21 anys', '22 anys', '23 anys', '24 anys'])) & (df_poblacio_edat_clean['sexe'] == 'dones')]

# Agrupar per 'comarca' i sumar els valors per a homes
df_16_24_homes_grouped = df_16_24_homes.groupby('comarca')['valor'].sum().reset_index()
df_16_24_homes_grouped.rename(columns={'valor': 'poblacio_homes_16_a_24_anys'}, inplace=True)

# Agrupar per 'comarca' i sumar els valors per a dones
df_16_24_dones_grouped = df_16_24_dones.groupby('comarca')['valor'].sum().reset_index()
df_16_24_dones_grouped.rename(columns={'valor': 'poblacio_dones_16_a_24_anys'}, inplace=True)

# Fusionar els DataFrames per obtenir la població total, homes i dones per comarca
df_poblacio_edat_final = df_16_24_total_grouped.merge(df_16_24_homes_grouped, on='comarca', how='left')
df_poblacio_edat_final = df_poblacio_edat_final.merge(df_16_24_dones_grouped, on='comarca', how='left')

In [72]:
df_poblacio_edat_final.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,alt camp,4201.0,2224.0,1977.0
1,alt emporda,13308.0,6800.0,6508.0
2,alt penedes,10482.0,5389.0,5093.0
3,alt urgell,1732.0,921.0,811.0
4,alta ribagorca,300.0,146.0,154.0


In [73]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_edat_final.insert(0, 'codi', df_poblacio_edat_final['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_edat_final = df_poblacio_edat_final.sort_values(by='codi').reset_index(drop=True)
# Convertir les columnes numériques a tipus int
df_poblacio_edat_final[['poblacio_total_16_a_24_anys', 'poblacio_homes_16_a_24_anys', 'poblacio_dones_16_a_24_anys']] = df_poblacio_edat_final[['poblacio_total_16_a_24_anys', 'poblacio_homes_16_a_24_anys', 'poblacio_dones_16_a_24_anys']].astype(int)
df_poblacio_edat_final.head()

,codi,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,01,alt camp,4201,2224,1977
1,02,alt emporda,13308,6800,6508
2,03,alt penedes,10482,5389,5093
3,04,alt urgell,1732,921,811
4,05,alta ribagorca,300,146,154


In [74]:
df_poblacio_edat_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   codi                         42 non-null     object
 1   comarca                      42 non-null     object
 2   poblacio_total_16_a_24_anys  42 non-null     int32 
 3   poblacio_homes_16_a_24_anys  42 non-null     int32 
 4   poblacio_dones_16_a_24_anys  42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [75]:
df_poblacio_edat_final.to_csv('clean_data/2021/df_poblacio_edat.csv', index=False)

#### Població Activa (16-24 anys)<a class="anchor" id="poblacio_activa"></a>

In [77]:
df_poblacio_activa = pd.read_csv('csv/2021/poblacio_activa_2021.csv', delimiter=';', skiprows=6)

In [78]:
df_poblacio_activa.head()

,comarca,De 16 a 19 anys,De 20 a 24 anys,De 25 a 29 anys,De 30 a 34 anys,De 35 a 39 anys,De 40 a 44 anys,De 45 a 49 anys,De 50 a 54 anys,De 55 a 59 anys,De 60 a 64 anys,65 anys o més,Total
0,Alt Camp,264,1242,1746,2058,2430,3264,3150,2937,2610,1635,300,21636
1,Alt Empordà,696,3696,5268,5994,7437,9441,9285,8415,7431,5082,1197,63945
2,Alt Penedès,654,2829,4044,4554,6051,8493,8739,7383,6087,3711,804,53352
3,Alt Urgell,84,462,735,789,882,1209,1200,1134,1083,798,153,8529
4,Alta Ribagorça,12,81,153,198,219,279,306,267,237,171,36,1959


In [79]:
# Netejar el DataFrame 'df_poblacio_activa' utilitzant la funció 'clean_dataframe'
df_poblacio_activa_clean = clean_dataframe(df_poblacio_activa)
df_poblacio_activa_clean.head()

,comarca,de_16_a_19_anys,de_20_a_24_anys,de_25_a_29_anys,de_30_a_34_anys,de_35_a_39_anys,de_40_a_44_anys,de_45_a_49_anys,de_50_a_54_anys,de_55_a_59_anys,de_60_a_64_anys,65_anys_o_mes,total
0,alt camp,264,1242,1746,2058,2430,3264,3150,2937,2610,1635,300,21636
1,alt emporda,696,3696,5268,5994,7437,9441,9285,8415,7431,5082,1197,63945
2,alt penedes,654,2829,4044,4554,6051,8493,8739,7383,6087,3711,804,53352
3,alt urgell,84,462,735,789,882,1209,1200,1134,1083,798,153,8529
4,alta ribagorca,12,81,153,198,219,279,306,267,237,171,36,1959


In [80]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_poblacio_activa_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya'], dtype=object)

In [81]:
# Filtrar les dades del DataFrame per obtenir només les comarques necesaries
df_poblacio_activa_clean = filtrar_comarques(df_poblacio_activa_clean, 'comarca')
df_poblacio_activa_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   comarca          42 non-null     object
 1   de_16_a_19_anys  42 non-null     int64 
 2   de_20_a_24_anys  42 non-null     int64 
 3   de_25_a_29_anys  42 non-null     int64 
 4   de_30_a_34_anys  42 non-null     int64 
 5   de_35_a_39_anys  42 non-null     int64 
 6   de_40_a_44_anys  42 non-null     int64 
 7   de_45_a_49_anys  42 non-null     int64 
 8   de_50_a_54_anys  42 non-null     int64 
 9   de_55_a_59_anys  42 non-null     int64 
 10  de_60_a_64_anys  42 non-null     int64 
 11  65_anys_o_mes    42 non-null     int64 
 12  total            42 non-null     int64 
dtypes: int64(12), object(1)
memory usage: 4.4+ KB


In [82]:
# Crear una nova columna 'poblacio_activa_16_a_24_anys' sumant les columnes 'de_16_a_19_anys' i 'de_20_a_24_anys'
df_poblacio_activa_clean.loc[:, 'poblacio_activa_16_a_24_anys'] = (
    df_poblacio_activa_clean['de_16_a_19_anys'] + df_poblacio_activa_clean['de_20_a_24_anys']
)
df_poblacio_activa_clean.head()

,comarca,de_16_a_19_anys,de_20_a_24_anys,de_25_a_29_anys,de_30_a_34_anys,de_35_a_39_anys,de_40_a_44_anys,de_45_a_49_anys,de_50_a_54_anys,de_55_a_59_anys,de_60_a_64_anys,65_anys_o_mes,total,poblacio_activa_16_a_24_anys
0,alt camp,264,1242,1746,2058,2430,3264,3150,2937,2610,1635,300,21636,1506
1,alt emporda,696,3696,5268,5994,7437,9441,9285,8415,7431,5082,1197,63945,4392
2,alt penedes,654,2829,4044,4554,6051,8493,8739,7383,6087,3711,804,53352,3483
3,alt urgell,84,462,735,789,882,1209,1200,1134,1083,798,153,8529,546
4,alta ribagorca,12,81,153,198,219,279,306,267,237,171,36,1959,93


In [83]:
# Seleccionar només les columnes 'comarca' i 'poblacio_activa_16_a_24_anys'
df_poblacio_activa = df_poblacio_activa_clean[['comarca','poblacio_activa_16_a_24_anys']]
df_poblacio_activa.head()

,comarca,poblacio_activa_16_a_24_anys
0,alt camp,1506
1,alt emporda,4392
2,alt penedes,3483
3,alt urgell,546
4,alta ribagorca,93


In [84]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_activa.insert(0, 'codi', df_poblacio_activa['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_activa = df_poblacio_activa.sort_values(by='codi').reset_index(drop=True)
df_poblacio_activa.head()

,codi,comarca,poblacio_activa_16_a_24_anys
0,01,alt camp,1506
1,02,alt emporda,4392
2,03,alt penedes,3483
3,04,alt urgell,546
4,05,alta ribagorca,93


In [85]:
# Fusionar els DataFrames 'df_poblacio_activa' i 'df_poblacio_genere_clean' utilitzant la columna comuna 'codi'
df_poblacio_activa_merged = df_poblacio_activa.merge(df_poblacio_genere_clean[['codi', 'total']], on='codi', how='left')
df_poblacio_activa_merged.head()

,codi,comarca,poblacio_activa_16_a_24_anys,total
0,01,alt camp,1506,45078
1,02,alt emporda,4392,143397
2,03,alt penedes,3483,110198
3,04,alt urgell,546,20444
4,05,alta ribagorca,93,3960


In [86]:
# Percentatge de població activa (16-24 anys) respecte al total
df_poblacio_activa_merged['pct_activa'] = (df_poblacio_activa_merged['poblacio_activa_16_a_24_anys'] / df_poblacio_activa_merged['total'] * 100).round(2)

# Ràtio de població activa (16-24 anys) respecte al total
df_poblacio_activa_merged['ratio_activa'] = (df_poblacio_activa_merged['poblacio_activa_16_a_24_anys'] / df_poblacio_activa_merged['total'])
df_poblacio_activa_merged.head()

,codi,comarca,poblacio_activa_16_a_24_anys,total,pct_activa,ratio_activa
0,01,alt camp,1506,45078,3.34,0.033409
1,02,alt emporda,4392,143397,3.06,0.030628
2,03,alt penedes,3483,110198,3.16,0.031607
3,04,alt urgell,546,20444,2.67,0.026707
4,05,alta ribagorca,93,3960,2.35,0.023485


In [87]:
df_poblacio_activa_merged.to_csv('clean_data/2021/df_poblacio_activa.csv', index=False)

#### Població Inactiva<a class="anchor" id="poblacio_inactiva"></a>

In [89]:
df_poblacio_inactiva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   comarca                                          55 non-null     object
 1   Població pensionista de jubilació, prejubilació  55 non-null     int64 
 2   Població pensionista d'invalidesa                55 non-null     int64 
 3   Població estudiant                               55 non-null     int64 
 4   Població amb una altra situació d'inactivitat    55 non-null     int64 
 5   Total població inactiva                          55 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.7+ KB


In [90]:
df_poblacio_inactiva.head()

,comarca,"Població pensionista de jubilació, prejubilació",Població pensionista d'invalidesa,Població estudiant,Població amb una altra situació d'inactivitat,Total població inactiva
0,Alt Camp,7412,1109,1994,5576,16091
1,Alt Empordà,20002,2157,5826,27485,55470
2,Alt Penedès,15768,2313,5149,14165,37395
3,Alt Urgell,3445,491,853,4243,9032
4,Alta Ribagorça,583,101,171,580,1435


In [91]:
# Netejar el DataFrame 'df_poblacio_inactiva' amb la funció 'clean_dataframe'
df_poblacio_inactiva_clean = clean_dataframe(df_poblacio_inactiva)
df_poblacio_inactiva_clean.head()

,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva
0,alt camp,7412,1109,1994,5576,16091
1,alt emporda,20002,2157,5826,27485,55470
2,alt penedes,15768,2313,5149,14165,37395
3,alt urgell,3445,491,853,4243,9032
4,alta ribagorca,583,101,171,580,1435


In [92]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_poblacio_inactiva_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [93]:
# Filtrar les comarques en el DataFrame netejat
df_poblacio_inactiva_clean = filtrar_comarques(df_poblacio_inactiva_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_poblacio_inactiva_clean['comarca'].unique())

42

In [94]:
df_poblacio_inactiva_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental'],
      dtype=object)

In [95]:
df_poblacio_inactiva_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   comarca                                          42 non-null     object
 1   poblacio_pensionista_de_jubilacio,_prejubilacio  42 non-null     int64 
 2   poblacio_pensionista_d'invalidesa                42 non-null     int64 
 3   poblacio_estudiant                               42 non-null     int64 
 4   poblacio_amb_una_altra_situacio_d'inactivitat    42 non-null     int64 
 5   total_poblacio_inactiva                          42 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.1+ KB


In [96]:
df_poblacio_inactiva_clean.head()

,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva
0,alt camp,7412,1109,1994,5576,16091
1,alt emporda,20002,2157,5826,27485,55470
2,alt penedes,15768,2313,5149,14165,37395
3,alt urgell,3445,491,853,4243,9032
4,alta ribagorca,583,101,171,580,1435


In [97]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_inactiva_clean.insert(0, 'codi', df_poblacio_inactiva_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_inactiva_clean = df_poblacio_inactiva_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_inactiva_clean.head()

,codi,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva
0,01,alt camp,7412,1109,1994,5576,16091
1,02,alt emporda,20002,2157,5826,27485,55470
2,03,alt penedes,15768,2313,5149,14165,37395
3,04,alt urgell,3445,491,853,4243,9032
4,05,alta ribagorca,583,101,171,580,1435


In [98]:
# Fusionar els DataFrames 'df_poblacio_inactiva_clean' i 'df_poblacio_genere_clean' utilitzant la columna comuna 'codi'
df_poblacio_inactiva_merged = df_poblacio_inactiva_clean.merge(df_poblacio_genere_clean[['codi', 'total']], on='codi', how='left')
df_poblacio_inactiva_merged.head()

,codi,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva,total
0,01,alt camp,7412,1109,1994,5576,16091,45078
1,02,alt emporda,20002,2157,5826,27485,55470,143397
2,03,alt penedes,15768,2313,5149,14165,37395,110198
3,04,alt urgell,3445,491,853,4243,9032,20444
4,05,alta ribagorca,583,101,171,580,1435,3960


In [99]:
# Percentatge de població inactiva respecte al total
df_poblacio_inactiva_merged['pct_atur'] = (df_poblacio_inactiva_clean['poblacio_amb_una_altra_situacio_d\'inactivitat'] / df_poblacio_inactiva_merged['total'] * 100).round(2)

# Ràtio de població inactiva respecte al total
df_poblacio_inactiva_merged['ratio_atur'] = (df_poblacio_inactiva_clean['poblacio_amb_una_altra_situacio_d\'inactivitat'] / df_poblacio_inactiva_merged['total'])
df_poblacio_inactiva_merged.head()

,codi,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva,total,pct_atur,ratio_atur
0,01,alt camp,7412,1109,1994,5576,16091,45078,12.37,0.123697
1,02,alt emporda,20002,2157,5826,27485,55470,143397,19.17,0.191671
2,03,alt penedes,15768,2313,5149,14165,37395,110198,12.85,0.128541
3,04,alt urgell,3445,491,853,4243,9032,20444,20.75,0.207543
4,05,alta ribagorca,583,101,171,580,1435,3960,14.65,0.146465


In [100]:
# Canviar el nom de la columna 'total' per 'total_poblacio'
df_poblacio_inactiva_merged.rename(columns={'total': 'total_poblacio'}, inplace=True)
df_poblacio_inactiva_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   codi                                             42 non-null     object 
 1   comarca                                          42 non-null     object 
 2   poblacio_pensionista_de_jubilacio,_prejubilacio  42 non-null     int64  
 3   poblacio_pensionista_d'invalidesa                42 non-null     int64  
 4   poblacio_estudiant                               42 non-null     int64  
 5   poblacio_amb_una_altra_situacio_d'inactivitat    42 non-null     int64  
 6   total_poblacio_inactiva                          42 non-null     int64  
 7   total_poblacio                                   42 non-null     int32  
 8   pct_atur                                         42 non-null     float64
 9   ratio_atur                        

In [101]:
df_poblacio_inactiva_merged['total_poblacio'].sum()

7749896

In [102]:
df_poblacio_inactiva_merged.to_csv('clean_data/2021/df_poblacio_inactiva.csv', index=False)

#### Atur per Edat<a class="anchor" id="atur"></a>

In [104]:
df_atur_edat = pd.read_csv('csv/2021/atur_edat_2021.csv', delimiter=';', skiprows=6)

In [105]:
df_atur_edat.head()

,comarca,16-24,25+,Total
0,Alt Camp,227.5,2695.5,2922.9
1,Alt Empordà,671.0,7752.0,8423.0
2,Alt Penedès,329.4,5556.2,5885.6
3,Alt Urgell,70.8,873.6,944.3
4,Alta Ribagorça,10.8,132.0,142.7


In [106]:
df_atur_edat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comarca  55 non-null     object 
 1   16-24    55 non-null     float64
 2   25+      55 non-null     float64
 3   Total    55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [107]:
# Netejar el DataFrame 'df_atur_edat' amb la funció 'clean_dataframe'
df_atur_edat_clean = clean_dataframe(df_atur_edat)
df_atur_edat_clean.head()

,comarca,16-24,25+,total
0,alt camp,227.5,2695.5,2922.9
1,alt emporda,671.0,7752.0,8423.0
2,alt penedes,329.4,5556.2,5885.6
3,alt urgell,70.8,873.6,944.3
4,alta ribagorca,10.8,132.0,142.7


In [108]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_edat_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [109]:
# Filtrar les comarques en el DataFrame netejat
df_atur_edat_clean = filtrar_comarques(df_atur_edat_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_edat_clean['comarca'].unique())

42

In [110]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_edat_clean.insert(0, 'codi', df_atur_edat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_edat_clean = df_atur_edat_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,227.5,2695.5,2922.9
1,02,alt emporda,671.0,7752.0,8423.0
2,03,alt penedes,329.4,5556.2,5885.6
3,04,alt urgell,70.8,873.6,944.3
4,05,alta ribagorca,10.8,132.0,142.7


In [111]:
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   codi     42 non-null     object 
 1   comarca  42 non-null     object 
 2   16-24    42 non-null     float64
 3   25+      42 non-null     float64
 4   total    42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [112]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_edat_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_edat_clean[columnes_float] = (
    np.ceil(df_atur_edat_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   16-24    42 non-null     int32 
 3   25+      42 non-null     int32 
 4   total    42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [113]:
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,228,2696,2923
1,02,alt emporda,671,7752,8423
2,03,alt penedes,330,5557,5886
3,04,alt urgell,71,874,945
4,05,alta ribagorca,11,132,143


In [114]:
df_atur_edat_clean.to_csv('clean_data/2021/df_atur_edat.csv', index=False)

#### Atur per Gènere<a class="anchor" id="atur_genere"></a>

In [377]:
df_atur_genere = pd.read_csv('csv/2021/atur_genere_2021.csv', delimiter=';', skiprows=6)

In [379]:
df_atur_genere.head()

,comarca,Total_homes,Total_dones,Total
0,Alt Camp,1286.1,1636.8,2922.9
1,Alt Empordà,3763.8,4659.3,8423.1
2,Alt Penedès,2513.1,3372.5,5885.6
3,Alt Urgell,411.7,532.7,944.4
4,Alta Ribagorça,67.9,74.8,142.7


In [381]:
df_atur_genere.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comarca      55 non-null     object 
 1   Total_homes  55 non-null     float64
 2   Total_dones  55 non-null     float64
 3   Total        55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [383]:
# Netejar el DataFrame 'df_atur_genere' amb la funció 'clean_dataframe'
df_atur_genere_clean = clean_dataframe(df_atur_genere)
df_atur_genere_clean.head()

,comarca,total_homes,total_dones,total
0,alt camp,1286.1,1636.8,2922.9
1,alt emporda,3763.8,4659.3,8423.1
2,alt penedes,2513.1,3372.5,5885.6
3,alt urgell,411.7,532.7,944.4
4,alta ribagorca,67.9,74.8,142.7


In [385]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_genere_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [387]:
# Filtrar les comarques en el DataFrame netejat
df_atur_genere_clean = filtrar_comarques(df_atur_genere_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_genere_clean['comarca'].unique())

42

In [389]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_genere_clean.insert(0, 'codi', df_atur_genere_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_genere_clean = df_atur_genere_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1286.1,1636.8,2922.9
1,02,alt emporda,3763.8,4659.3,8423.1
2,03,alt penedes,2513.1,3372.5,5885.6
3,04,alt urgell,411.7,532.7,944.4
4,05,alta ribagorca,67.9,74.8,142.7


In [391]:
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   codi         42 non-null     object 
 1   comarca      42 non-null     object 
 2   total_homes  42 non-null     float64
 3   total_dones  42 non-null     float64
 4   total        42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [393]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_genere_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_genere_clean[columnes_float] = (
    np.ceil(df_atur_genere_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   codi         42 non-null     object
 1   comarca      42 non-null     object
 2   total_homes  42 non-null     int32 
 3   total_dones  42 non-null     int32 
 4   total        42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [395]:
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1287,1637,2923
1,02,alt emporda,3764,4660,8424
2,03,alt penedes,2514,3373,5886
3,04,alt urgell,412,533,945
4,05,alta ribagorca,68,75,143


In [397]:
df_atur_genere_clean.to_csv('clean_data/2021/df_atur_genere.csv', index=False)

#### Percentatge/Ratio Estudiants<a class="anchor" id="pct_estudiants"></a>

In [128]:
df_poblacio_inactiva_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   codi                                             42 non-null     object 
 1   comarca                                          42 non-null     object 
 2   poblacio_pensionista_de_jubilacio,_prejubilacio  42 non-null     int64  
 3   poblacio_pensionista_d'invalidesa                42 non-null     int64  
 4   poblacio_estudiant                               42 non-null     int64  
 5   poblacio_amb_una_altra_situacio_d'inactivitat    42 non-null     int64  
 6   total_poblacio_inactiva                          42 non-null     int64  
 7   total_poblacio                                   42 non-null     int32  
 8   pct_atur                                         42 non-null     float64
 9   ratio_atur                        

In [129]:
# Crear un nou DataFrame amb les columnes 'codi', 'comarca', 'poblacio_estudiant' i 'total_poblacio'
df_pct_ratio_estudiants = df_poblacio_inactiva_merged[['codi', 'comarca', 'poblacio_estudiant', 'total_poblacio']].copy()

# Calcular el percentatge d'estudiants respecte al total de població
df_pct_ratio_estudiants['pct_estudiants'] = (df_poblacio_inactiva_merged['poblacio_estudiant'] / df_poblacio_inactiva_merged['total_poblacio'] * 100).round(2)

# Calcular la ràtio d'estudiants respecte al total de població
df_pct_ratio_estudiants['ratio_estudiants'] = (df_poblacio_inactiva_merged['poblacio_estudiant'] / df_poblacio_inactiva_merged['total_poblacio'])
df_pct_ratio_estudiants.head()

,codi,comarca,poblacio_estudiant,total_poblacio,pct_estudiants,ratio_estudiants
0,01,alt camp,1994,45078,4.42,0.044234
1,02,alt emporda,5826,143397,4.06,0.040628
2,03,alt penedes,5149,110198,4.67,0.046725
3,04,alt urgell,853,20444,4.17,0.041724
4,05,alta ribagorca,171,3960,4.32,0.043182


In [130]:
df_pct_ratio_estudiants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   codi                42 non-null     object 
 1   comarca             42 non-null     object 
 2   poblacio_estudiant  42 non-null     int64  
 3   total_poblacio      42 non-null     int32  
 4   pct_estudiants      42 non-null     float64
 5   ratio_estudiants    42 non-null     float64
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 1.9+ KB


In [131]:
df_pct_ratio_estudiants.to_csv('clean_data/2021/df_estudiants_perc_ratio.csv', index=False)

#### Població per Nacionalitat<a class="anchor" id="poblacio_nacionalitat"></a>

In [133]:
df_poblacio_nacionalitat = pd.read_csv('csv/2021/poblacio_nacionalitat_2021.csv', delimiter=';', skiprows=7)

In [134]:
df_poblacio_nacionalitat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   comarca                     55 non-null     object
 1   Espanyola                   55 non-null     int64 
 2   Resta UE                    55 non-null     int64 
 3   Resta d'Europa              55 non-null     int64 
 4   Àfrica                      55 non-null     int64 
 5   Amèrica del Nord i Central  55 non-null     int64 
 6   Amèrica del Sud             55 non-null     int64 
 7   Àsia i Oceania              55 non-null     int64 
 8   Total                       55 non-null     int64 
dtypes: int64(8), object(1)
memory usage: 4.0+ KB


In [135]:
df_poblacio_nacionalitat.head()

,comarca,Espanyola,Resta UE,Resta d'Europa,Àfrica,Amèrica del Nord i Central,Amèrica del Sud,Àsia i Oceania,Total
0,Alt Camp,39062,1310,502,2989,229,702,284,45078
1,Alt Empordà,108103,11083,2766,13636,2481,4201,1127,143397
2,Alt Penedès,98205,1811,769,5958,680,1935,840,110198
3,Alt Urgell,17788,971,314,446,164,634,127,20444
4,Alta Ribagorça,3455,177,56,147,35,84,6,3960


In [136]:
# Netejar el DataFrame 'df_poblacio_nacionalitat' amb la funció 'clean_dataframe'
df_poblacio_nacionalitat_clean = clean_dataframe(df_poblacio_nacionalitat)
# Filtrar les comarques en el DataFrame netejat
df_poblacio_nacionalitat_clean = filtrar_comarques(df_poblacio_nacionalitat_clean, 'comarca')
df_poblacio_nacionalitat_clean.head()

,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,alt camp,39062,1310,502,2989,229,702,284,45078
1,alt emporda,108103,11083,2766,13636,2481,4201,1127,143397
2,alt penedes,98205,1811,769,5958,680,1935,840,110198
3,alt urgell,17788,971,314,446,164,634,127,20444
4,alta ribagorca,3455,177,56,147,35,84,6,3960


In [137]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_nacionalitat_clean.insert(0, 'codi', df_poblacio_nacionalitat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_nacionalitat_clean = df_poblacio_nacionalitat_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,01,alt camp,39062,1310,502,2989,229,702,284,45078
1,02,alt emporda,108103,11083,2766,13636,2481,4201,1127,143397
2,03,alt penedes,98205,1811,769,5958,680,1935,840,110198
3,04,alt urgell,17788,971,314,446,164,634,127,20444
4,05,alta ribagorca,3455,177,56,147,35,84,6,3960


In [138]:
# Llista de nacionalitats
nacionalitats = ['espanyola', 'resta_ue', 'resta_d\'europa', 'africa', 'america_del_nord_i_central', 'america_del_sud', 'asia_i_oceania']

# Calcular el percentatge i la ràtio per a cada nacionalitat
for nacionalitat in nacionalitats:
    # Percentatge respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_pct'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total'] * 100
    ).round(2)
    
    # Ràtio respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_ratio'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total']
    )

# Mostrar les primeres files del DataFrame resultant
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total,...,resta_d'europa_pct,resta_d'europa_ratio,africa_pct,africa_ratio,america_del_nord_i_central_pct,america_del_nord_i_central_ratio,america_del_sud_pct,america_del_sud_ratio,asia_i_oceania_pct,asia_i_oceania_ratio
0,01,alt camp,39062,1310,502,2989,229,702,284,45078,...,1.11,0.011136,6.63,0.066307,0.51,0.005080,1.56,0.015573,0.63,0.006300
1,02,alt emporda,108103,11083,2766,13636,2481,4201,1127,143397,...,1.93,0.019289,9.51,0.095093,1.73,0.017302,2.93,0.029296,0.79,0.007859
2,03,alt penedes,98205,1811,769,5958,680,1935,840,110198,...,0.70,0.006978,5.41,0.054066,0.62,0.006171,1.76,0.017559,0.76,0.007623
3,04,alt urgell,17788,971,314,446,164,634,127,20444,...,1.54,0.015359,2.18,0.021816,0.80,0.008022,3.10,0.031012,0.62,0.006212
4,05,alta ribagorca,3455,177,56,147,35,84,6,3960,...,1.41,0.014141,3.71,0.037121,0.88,0.008838,2.12,0.021212,0.15,0.001515


In [139]:
df_poblacio_nacionalitat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   codi                              42 non-null     object 
 1   comarca                           42 non-null     object 
 2   espanyola                         42 non-null     int64  
 3   resta_ue                          42 non-null     int64  
 4   resta_d'europa                    42 non-null     int64  
 5   africa                            42 non-null     int64  
 6   america_del_nord_i_central        42 non-null     int64  
 7   america_del_sud                   42 non-null     int64  
 8   asia_i_oceania                    42 non-null     int64  
 9   total                             42 non-null     int64  
 10  espanyola_pct                     42 non-null     float64
 11  espanyola_ratio                   42 non-null     float64
 12  resta_ue_p

In [140]:
df_poblacio_nacionalitat_clean.to_csv('clean_data/2021/df_poblacio_nacionalitat.csv', index=False)

### Dades Socioeconòmiques<a class="anchor" id="dades_socioeconomiques"></a>

#### Repetidors<a class="anchor" id="repetidors"></a>

https://educacio.gencat.cat/ca/departament/estadistiques/estadistiques-ensenyament/cursos-anteriors/curs-2021-2022/eso/

In [144]:
df_repetidors = pd.read_csv('csv/2021/alumnes_repetidors_2021.csv', delimiter=';')

In [145]:
df_repetidors.head()

,Curs,Estudi,Codi àrea territorial,Àrea territorial,Codi comarca,Comarca,Codi municipi,Municipi,Codi districte,Naturalesa,Titularitat,Nivell,Sexe,Concert,Resultat de l'avaluació,Nombre alumnes
0,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,DONA,SI,Alumnes avaluats,3
1,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,DONA,SI,Promocionen al mes de juny,2
2,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,DONA,SI,Promocionen al mes de setembre,1
3,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,HOME,SI,Alumnes avaluats,7
4,2020/2021,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,ASSOCIACIONS,1,HOME,SI,Promocionen al mes de juny,3


In [146]:
df_repetidors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24780 entries, 0 to 24779
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Curs                     24780 non-null  object
 1   Estudi                   24780 non-null  object
 2   Codi àrea territorial    24780 non-null  int64 
 3   Àrea territorial         24780 non-null  object
 4   Codi comarca             24780 non-null  int64 
 5   Comarca                  24780 non-null  object
 6   Codi municipi            24780 non-null  int64 
 7   Municipi                 24780 non-null  object
 8   Codi districte           24780 non-null  int64 
 9   Naturalesa               24780 non-null  object
 10  Titularitat              24780 non-null  object
 11  Nivell                   24780 non-null  int64 
 12  Sexe                     24780 non-null  object
 13  Concert                  9312 non-null   object
 14  Resultat de l'avaluació  24780 non-nul

In [147]:
df_repetidors['Comarca'].unique()

array(['BARCELONÈS', 'ALT EMPORDÀ', 'BAIX EMPORDÀ', 'CERDANYA',
       'GARROTXA', 'GIRONÈS', "PLA DE L'ESTANY", 'RIPOLLÈS', 'SELVA',
       'ALT URGELL', 'ALTA RIBAGORÇA', 'GARRIGUES', 'NOGUERA',
       'PALLARS JUSSÀ', 'PALLARS SOBIRÀ', "PLA D'URGELL", 'SEGARRA',
       'SEGRIÀ', 'URGELL', "VAL D'ARAN", 'ALT CAMP', 'BAIX CAMP',
       'BAIX PENEDÈS', 'CONCA DE BARBERÀ', 'PRIORAT', 'TARRAGONÈS',
       'ALT PENEDÈS', 'GARRAF', 'BAIX EBRE', 'MONTSIÀ', "RIBERA D'EBRE",
       'TERRA ALTA', 'BAIX LLOBREGAT', 'VALLÈS OCCIDENTAL', 'MARESME',
       'VALLÈS ORIENTAL', 'ANOIA', 'BAGES', 'BERGUEDÀ', 'OSONA',
       'SOLSONÈS', 'MOIANÈS'], dtype=object)

In [148]:
# Netejar el DataFrame 'df_repetidors' utilitzant la funció 'clean_dataframe'
df_repetidors_clean = clean_dataframe(df_repetidors)
df_repetidors_clean.head()

,curs,estudi,codi_area_territorial,area_territorial,codi_comarca,comarca,codi_municipi,municipi,codi_districte,naturalesa,titularitat,nivell,sexe,concert,resultat_de_l'avaluacio,nombre_alumnes
0,2020/2021,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,1,dona,si,alumnes avaluats,3
1,2020/2021,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,1,dona,si,promocionen al mes de juny,2
2,2020/2021,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,1,dona,si,promocionen al mes de setembre,1
3,2020/2021,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,1,home,si,alumnes avaluats,7
4,2020/2021,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,associacions,1,home,si,promocionen al mes de juny,3


In [149]:
# Filtrar les dades del DataFrame per obtenir només les comarques rellevants
df_repetidors_clean = filtrar_comarques(df_repetidors_clean, 'comarca')

In [150]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
display(df_repetidors_clean['comarca'].unique())
print(len(df_repetidors_clean['comarca'].unique()))

array(['barcelones', 'alt emporda', 'baix emporda', 'cerdanya',
       'garrotxa', 'girones', "pla de l'estany", 'ripolles', 'selva',
       'alt urgell', 'alta ribagorca', 'garrigues', 'noguera',
       'pallars jussa', 'pallars sobira', "pla d'urgell", 'segarra',
       'segria', 'urgell', 'alt camp', 'baix camp', 'baix penedes',
       'conca de barbera', 'priorat', 'tarragones', 'alt penedes',
       'garraf', 'baix ebre', 'montsia', "ribera d'ebre", 'terra alta',
       'baix llobregat', 'valles occidental', 'maresme',
       'valles oriental', 'anoia', 'bages', 'bergueda', 'osona',
       'solsones', 'moianes'], dtype=object)

41


In [151]:
# Obtenir els valors únics de la columna 'resultat_de_l'avaluacio' en el DataFrame netejat
df_repetidors_clean['resultat_de_l\'avaluacio'].unique()

array(['alumnes avaluats', 'promocionen al mes de juny',
       'promocionen al mes de setembre', 'promocionen amb pendents',
       "s'ha expedit certificat", 'no promocionen'], dtype=object)

In [152]:
# Filtrar el DataFrame per mantenir només les files amb 'resultat_de_l'avaluacio' igual a 'no promocionen'
df_no_promocionen = df_repetidors_clean[df_repetidors_clean['resultat_de_l\'avaluacio'] == 'no promocionen']

# Agrupar les dades per 'comarca' i 'sexe', sumant els valors de 'nombre_alumnes'
df_repetidors = df_no_promocionen.groupby(['comarca', 'sexe'])['nombre_alumnes'].sum().reset_index()

# Renombrar la columna 'nombre_alumnes' a 'repetixen'
df_repetidors = df_repetidors.rename(columns={'nombre_alumnes': 'repetixen'})

# Mostrar les primeres files del DataFrame resultant
df_repetidors.head()

,comarca,sexe,repetixen
0,alt camp,dona,52
1,alt camp,home,85
2,alt emporda,dona,115
3,alt emporda,home,201
4,alt penedes,dona,53


In [153]:
# Afegim la columna 'codi'  per fer match amb els noms de les comarques
df_repetidors.insert(0, 'codi', df_repetidors['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_repetidors = df_repetidors.sort_values(by='codi').reset_index(drop=True)
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,52
1,01,alt camp,home,85
2,02,alt emporda,dona,115
3,02,alt emporda,home,201
4,03,alt penedes,dona,53


In [154]:
df_repetidors.to_csv('clean_data/2021/df_repetidors.csv', index=False)

### Total Repetidors per Comarca<a class="anchor" id="repetidors_comarca"></a>

In [156]:
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,52
1,01,alt camp,home,85
2,02,alt emporda,dona,115
3,02,alt emporda,home,201
4,03,alt penedes,dona,53


In [157]:
# Agrupar les dades per 'codi' i 'comarca', sumant els valors de 'repetixen'
df_total_repetidors = df_repetidors.groupby(['codi', 'comarca'])['repetixen'].sum().reset_index()

# Renombrar la columna 'repetixen' a 'total_repetixen'
df_total_repetidors.rename(columns={'repetixen': 'total_repetixen'}, inplace=True)
df_total_repetidors.head()

,codi,comarca,total_repetixen
0,01,alt camp,137
1,02,alt emporda,316
2,03,alt penedes,192
3,04,alt urgell,44
4,05,alta ribagorca,10


In [158]:
df_pct_ratio_estudiants.head()

,codi,comarca,poblacio_estudiant,total_poblacio,pct_estudiants,ratio_estudiants
0,01,alt camp,1994,45078,4.42,0.044234
1,02,alt emporda,5826,143397,4.06,0.040628
2,03,alt penedes,5149,110198,4.67,0.046725
3,04,alt urgell,853,20444,4.17,0.041724
4,05,alta ribagorca,171,3960,4.32,0.043182


In [159]:
# Fusionar els DataFrames 'df_total_repetidors' i 'df_pct_ratio_estudiants' utilitzant la columna comuna 'codi'
df_total_repetidors_merged = df_total_repetidors.merge(df_pct_ratio_estudiants[['codi', 'poblacio_estudiant']], on='codi', how='left')

# Seleccionar només les columnes necessàries del DataFrame resultant
df_total_repetidors_merged = df_total_repetidors_merged[['codi', 'comarca', 'total_repetixen', 'poblacio_estudiant']].copy()
df_total_repetidors_merged.head()

,codi,comarca,total_repetixen,poblacio_estudiant
0,01,alt camp,137,1994
1,02,alt emporda,316,5826
2,03,alt penedes,192,5149
3,04,alt urgell,44,853
4,05,alta ribagorca,10,171


In [160]:
# Calcular el percentatge de repetidors respecte a la població estudiant
df_total_repetidors_merged['pct_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'] * 100).round(2)

# Calcular la ràtio de repetidors respecte a la població estudiant
df_total_repetidors_merged['ratio_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'])
df_total_repetidors_merged.head()

,codi,comarca,total_repetixen,poblacio_estudiant,pct_repetidors,ratio_repetidors
0,01,alt camp,137,1994,6.87,0.068706
1,02,alt emporda,316,5826,5.42,0.054240
2,03,alt penedes,192,5149,3.73,0.037289
3,04,alt urgell,44,853,5.16,0.051583
4,05,alta ribagorca,10,171,5.85,0.058480


In [161]:
df_total_repetidors_merged.to_csv('clean_data/2021/df_total_repetidors.csv', index=False)

### Repetidors Població<a class="anchor" id="repetidors_poblacio"></a>

In [163]:
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,52
1,01,alt camp,home,85
2,02,alt emporda,dona,115
3,02,alt emporda,home,201
4,03,alt penedes,dona,53


In [164]:
df_total_repetidors_merged.head()

,codi,comarca,total_repetixen,poblacio_estudiant,pct_repetidors,ratio_repetidors
0,01,alt camp,137,1994,6.87,0.068706
1,02,alt emporda,316,5826,5.42,0.054240
2,03,alt penedes,192,5149,3.73,0.037289
3,04,alt urgell,44,853,5.16,0.051583
4,05,alta ribagorca,10,171,5.85,0.058480


In [165]:
# Fusionar 'df_repetidors' amb 'df_total_repetidors_merged' per afegir 'comarca' i 'poblacio_estudiant'
df_repetidors_genere_merged = df_repetidors.merge(
    df_total_repetidors_merged[['codi', 'comarca', 'poblacio_estudiant']], 
    on='codi', 
    how='left'
)
df_repetidors_genere_merged.head()

,codi,comarca_x,sexe,repetixen,comarca_y,poblacio_estudiant
0,01,alt camp,dona,52,alt camp,1994
1,01,alt camp,home,85,alt camp,1994
2,02,alt emporda,dona,115,alt emporda,5826
3,02,alt emporda,home,201,alt emporda,5826
4,03,alt penedes,dona,53,alt penedes,5149


In [166]:
# Eliminar 'comarca_y' i renombrar 'comarca_x' a 'comarca'
df_repetidors_genere_merged = df_repetidors_genere_merged.drop(columns=['comarca_y']).rename(columns={'comarca_x': 'comarca'})
df_repetidors_genere_merged.head()

,codi,comarca,sexe,repetixen,poblacio_estudiant
0,01,alt camp,dona,52,1994
1,01,alt camp,home,85,1994
2,02,alt emporda,dona,115,5826
3,02,alt emporda,home,201,5826
4,03,alt penedes,dona,53,5149


In [167]:
# Crear df_dones (mujeres) i df_homes (hombres) separats por genere
df_dones = df_repetidors_genere_merged[df_repetidors_genere_merged['sexe'] == 'dona']
df_homes = df_repetidors_genere_merged[df_repetidors_genere_merged['sexe'] == 'home']

display(df_dones.head())
display(df_homes.head())

,codi,comarca,sexe,repetixen,poblacio_estudiant
0,01,alt camp,dona,52,1994
2,02,alt emporda,dona,115,5826
4,03,alt penedes,dona,53,5149
6,04,alt urgell,dona,10,853
8,05,alta ribagorca,dona,4,171


,codi,comarca,sexe,repetixen,poblacio_estudiant
1,01,alt camp,home,85,1994
3,02,alt emporda,home,201,5826
5,03,alt penedes,home,139,5149
7,04,alt urgell,home,34,853
9,05,alta ribagorca,home,6,171


In [168]:
# Calcular el percentatge de repetidors per dones (mujeres)
df_dones = df_dones.copy()
df_homes = df_homes.copy()
df_dones.loc[:, 'pct_repetidors_dones'] = (df_dones['repetixen'] / df_dones['poblacio_estudiant']) * 100
df_dones.loc[:, 'ratio_repetidors_dones'] = df_dones['repetixen'] / df_dones['poblacio_estudiant']

# Calcular el percentatge de repetidors per homes (hombres)
df_homes.loc[:, 'pct_repetidors_homes'] = (df_homes['repetixen'] / df_homes['poblacio_estudiant']) * 100
df_homes.loc[:, 'ratio_repetidors_homes'] = df_homes['repetixen'] / df_homes['poblacio_estudiant']

# Unir els resultats de dones i homes en un sol DataFrame
df_repetidors_poblacio = pd.concat([df_dones, df_homes])

# Agrupar per 'codi' i 'comarca' per calcular els totals per comarca
df_repetidors_poblacio = df_repetidors_poblacio.groupby(['codi', 'comarca']).agg({
    'repetixen': 'sum',  
    'pct_repetidors_dones': 'mean',  
    'ratio_repetidors_dones': 'mean',  
    'pct_repetidors_homes': 'mean',  
    'ratio_repetidors_homes': 'mean' 
}).reset_index()

df_repetidors_poblacio.head()

,codi,comarca,repetixen,pct_repetidors_dones,ratio_repetidors_dones,pct_repetidors_homes,ratio_repetidors_homes
0,01,alt camp,137,2.607823,0.026078,4.262788,0.042628
1,02,alt emporda,316,1.973910,0.019739,3.450051,0.034501
2,03,alt penedes,192,1.029326,0.010293,2.699553,0.026996
3,04,alt urgell,44,1.172333,0.011723,3.985932,0.039859
4,05,alta ribagorca,10,2.339181,0.023392,3.508772,0.035088


In [169]:
# Renombrar la columna 'repetixen' a 'total_repetixen'
df_repetidors_poblacio = df_repetidors_poblacio.rename(columns={'repetixen': 'total_repetixen'})

df_repetidors_poblacio.head()

,codi,comarca,total_repetixen,pct_repetidors_dones,ratio_repetidors_dones,pct_repetidors_homes,ratio_repetidors_homes
0,01,alt camp,137,2.607823,0.026078,4.262788,0.042628
1,02,alt emporda,316,1.973910,0.019739,3.450051,0.034501
2,03,alt penedes,192,1.029326,0.010293,2.699553,0.026996
3,04,alt urgell,44,1.172333,0.011723,3.985932,0.039859
4,05,alta ribagorca,10,2.339181,0.023392,3.508772,0.035088


In [170]:
df_repetidors_poblacio.to_csv('clean_data/2021/df_repetidors_poblacio.csv', index=False)

### Nivell Socioeconòmic (Renda Familiar)<a class="anchor" id="renta_familiar"></a>

In [172]:
df_renta_familiar.head()

,Unnamed: 0,RFDB (miles de euros),RFDB por habitante (euros),RFDB por habitante (Índex Catalunya=100 por habitante)
0,Alt Camp,747053,16512,"90,0"
1,Alt Empordà,2059702,14339,"78,1"
2,Alt Penedès,1958312,17716,"96,5"
3,Alt Urgell,293835,14416,"78,5"
4,Alta Ribagorça,60948,15345,"83,6"


In [173]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_renta_familiar = df_renta_familiar.rename(columns={'Unnamed: 0': 'comarca'})
df_renta_familiar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   comarca                                                 51 non-null     object
 1   RFDB (miles de euros)                                   51 non-null     int64 
 2   RFDB por habitante (euros)                              51 non-null     int64 
 3   RFDB por habitante (Índex Catalunya=100 por habitante)  51 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.7+ KB


In [174]:
# Netejar el DataFrame 'df_renta_familiar' utilitzant la funció 'clean_dataframe'
df_renta_familiar_clean = clean_dataframe(df_renta_familiar)
# Filtrar les dades per obtenir només les comarques rellevants
df_renta_familiar_clean = filtrar_comarques(df_renta_familiar_clean, 'comarca')
df_renta_familiar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   comarca                                                 42 non-null     object
 1   rfdb_(miles_de_euros)                                   42 non-null     int64 
 2   rfdb_por_habitante_(euros)                              42 non-null     int64 
 3   rfdb_por_habitante_(index_catalunya=100_por_habitante)  42 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.4+ KB


In [175]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_renta_familiar_clean.insert(0, 'codi', df_renta_familiar_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_renta_familiar_clean = df_renta_familiar_clean.sort_values(by='codi').reset_index(drop=True)
df_renta_familiar_clean.head()

,codi,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,01,alt camp,747053,16512,"90,0"
1,02,alt emporda,2059702,14339,"78,1"
2,03,alt penedes,1958312,17716,"96,5"
3,04,alt urgell,293835,14416,"78,5"
4,05,alta ribagorca,60948,15345,"83,6"


In [176]:
# Convertir els valors de la columna a format numèric
df_renta_familiar_clean['rfdb_por_habitante_(index_catalunya=100_por_habitante)'] = (
    df_renta_familiar_clean['rfdb_por_habitante_(index_catalunya=100_por_habitante)']
    .str.replace(',', '.')  # Substituir les comes per punts
    .astype(float)          # Convertir els valors a tipus float
)
df_renta_familiar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   codi                                                    42 non-null     object 
 1   comarca                                                 42 non-null     object 
 2   rfdb_(miles_de_euros)                                   42 non-null     int64  
 3   rfdb_por_habitante_(euros)                              42 non-null     int64  
 4   rfdb_por_habitante_(index_catalunya=100_por_habitante)  42 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ KB


In [177]:
df_renta_familiar_clean.head()

,codi,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,01,alt camp,747053,16512,90.0
1,02,alt emporda,2059702,14339,78.1
2,03,alt penedes,1958312,17716,96.5
3,04,alt urgell,293835,14416,78.5
4,05,alta ribagorca,60948,15345,83.6


In [178]:
df_renta_familiar_clean.to_csv('clean_data/2021/df_renta_familiar.csv', index=False)

### Nivell Socioeconòmic (RFBD)<a class="anchor" id="rfbd"></a>

In [180]:
df_rfbd.head()

,Unnamed: 0,Recursos. Remuneración asalariados,Recursos. Excedente bruto explotación,Recursos. Prestaciones sociales,Usos. Cotizaciones sociales,Usos. Impuestos
0,Alt Camp,593173,173872,240921,179399,96508
1,Alt Empordà,1583386,657018,598935,512052,329330
2,Alt Penedès,1652972,474524,592603,490413,277769
3,Alt Urgell,197416,91423,97949,63921,36509
4,Alta Ribagorça,42647,22887,18521,14839,8501


In [181]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_rfbd = df_rfbd.rename(columns={'Unnamed: 0': 'comarca'})
df_rfbd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   comarca                                51 non-null     object
 1   Recursos. Remuneración asalariados     51 non-null     int64 
 2   Recursos. Excedente bruto explotación  51 non-null     int64 
 3   Recursos. Prestaciones sociales        51 non-null     int64 
 4   Usos. Cotizaciones sociales            51 non-null     int64 
 5   Usos. Impuestos                        51 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.5+ KB


In [182]:
# Netejar el DataFrame 'df_renta_familiar' utilitzant la funció 'clean_dataframe'
df_rfbd_clean = clean_dataframe(df_rfbd)
# Filtrar les dades per obtenir només les comarques rellevants
df_rfbd_clean = filtrar_comarques(df_rfbd_clean, 'comarca')
df_rfbd_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   comarca                                42 non-null     object
 1   recursos._remuneracion_asalariados     42 non-null     int64 
 2   recursos._excedente_bruto_explotacion  42 non-null     int64 
 3   recursos._prestaciones_sociales        42 non-null     int64 
 4   usos._cotizaciones_sociales            42 non-null     int64 
 5   usos._impuestos                        42 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.1+ KB


In [183]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_rfbd_clean.insert(0, 'codi', df_rfbd_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_rfbd_clean = df_rfbd_clean.sort_values(by='codi').reset_index(drop=True)
df_rfbd_clean.head()

,codi,comarca,recursos._remuneracion_asalariados,recursos._excedente_bruto_explotacion,recursos._prestaciones_sociales,usos._cotizaciones_sociales,usos._impuestos
0,01,alt camp,593173,173872,240921,179399,96508
1,02,alt emporda,1583386,657018,598935,512052,329330
2,03,alt penedes,1652972,474524,592603,490413,277769
3,04,alt urgell,197416,91423,97949,63921,36509
4,05,alta ribagorca,42647,22887,18521,14839,8501


In [184]:
df_rfbd_clean.to_csv('clean_data/2021/df_rfbd.csv', index=False)